In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from tqdm import tqdm_notebook as tqdm
from os import listdir
from os.path import isfile, join
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from pathlib import Path
import multiprocessing as mp

def makedir(path):
    Path(path).mkdir(parents=True, exist_ok=True)

def majority_vote(x):
    c = Counter(x)
    value, count = c.most_common()[0]
    return value
def nothing(x):
    if 1 != len(np.unique(x.values)):
        assert False, "error on aggregation"
    else:
        return x.values[0]
import itertools
from operator import itemgetter
    
topics_to_consider = ['402', '403', '405', '407', '408', '410', '415', '416', '418', '420', '421', '427', '428', '431', '440', '442', '445', '448']
topics_to_consider_moffat = ['402', '403', '405', '407', '408', '415', '416', '431', '440']
#topics_to_consider = topics_to_consider_moffat[:]

In [ ]:
import json
import pickle

def save_json(obj, path):
    with open(path, "w") as f:
        json.dump(obj, f, indent=4)

def load_json(path):
    with open(path, "r") as f:
        data = json.load(f)
    return data

def save_pickle(obj, path):
    with open(path, "wb") as f:
        pickle.dump(obj, f)

def load_pickle(path):
    with open(path, "rb") as f:
        data = pickle.load(f)
    return data
import os.path
def file_exists(fname):
    return os.path.isfile(fname) 

def makedir(PATH):
    from pathlib import Path
    Path(PATH).mkdir(parents=True, exist_ok=True)

def rmdir(PATH):
    shutil.rmtree(PATH, ignore_errors=True)
    
import argparse


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-k", type=int, default=3, nargs='+' )
parser.add_argument("-b", type=int, default=10, nargs='+')

parser.add_argument("--which", type=str, default='',choices=['all', 'adjacent', 'alleverysystem'], help='which pairs to rank') 
parser.add_argument("--rank", type=str, default='',choices=['none', 'avgrank', 'borda','rrf','combMNZ','avgrankreversed'], help='how to rank pairs' )
parser.add_argument("--preference", type=str, default='',choices=['S100', 'ME', 'S2', 'S4', 'moffatratio','S100toPref','MEtoPref', 'S2toPref', 'S4toPref', 'TREC8'], help='how to evaluate pairs' )
parser.add_argument("--topics", type=str, default='',choices=['all', 'subsetmoffat'], help='which topics to use' )  
parser.add_argument("--linearize", type=str, default='',choices=['sortbywin', 'btl','btlscores'], help='how to linearize preferences' ) 
parser.add_argument("--server", type=str, default='',choices=['mitel', 'bernini', 'local', 'gianluca'], help='where to run' ) 
parser.add_argument("--baseline", type=int, default=0 ,choices=[0,1], help='is a baseline?' ) 


stt_example = "--which all --rank borda --preference TREC8 --topics all --linearize sortbywin --server local -k 10 -b 3400".split()
#stt_example = "--which adjacent --rank none --preference S100 --topics all --linearize sortbywin --server local -k 10 -b 10000 ".split()

line_args = parser.parse_args(stt_example)
#line_args = parser.parse_args()

ARG_budgets = line_args.b
ARG_ks = line_args.k
ARG_which = line_args.which
ARG_rank = line_args.rank
ARG_preference = line_args.preference
ARG_topics = line_args.topics
ARG_linearize = line_args.linearize
ARG_server = line_args.server
ARG_baseline = line_args.baseline


budget = ARG_budgets[0]
k = ARG_ks[0]
print(line_args)
# for budget in ARG_budgets:
#     for k in ARG_ks:
#         print(f"{budget} -- {k}")

In [ ]:
if ARG_server=='mitel':
    UNCOMPRESSED_PATH = '/mnt/sda/kevinr/IR/Uncompressed/AH99/input/'
    QRELS_PATH = '/mnt/sda/kevinr/IR/'
    DICT_AT_PATH = '/mnt/sda/kevinr/preference-on-a-budget/pers_script/dict_at/' 
    TREC_EVAL_PATH = '../IR/trec_eval-master/trec_eval'
    DFIR_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/run_as_pickles/overcomplex/AH99.pickle'
    if ARG_preference=='S100':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/S100.csv'
    elif ARG_preference=='ME':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/ME.csv'
    elif ARG_preference=='S2':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/S2.csv'
    elif ARG_preference=='S4':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/S4.csv'
    elif ARG_preference=='moffatratio':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/qrel_pref+rele+ratio-KRNormalized.csv'
    elif ARG_preference=='S100toPref':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/S100toPreference/S100_pairwise_full_scores.csv'
    elif ARG_preference=='MEtoPref':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/S100toPreference/ME_pairwise_full_scores.csv'
    elif ARG_preference=='S2toPref':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/S100toPreference/S2_pairwise_full_scores.csv'
    elif ARG_preference=='S4toPref':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/S100toPreference/S4_pairwise_full_scores.csv'
    elif ARG_preference=='TREC8':
        DFCROWD_PATH = '/mnt/sda/kevinr/preference-on-a-budget/src/S100.csv'
    
elif ARG_server=='bernini':
    UNCOMPRESSED_PATH = '/home/kevinr/IR/Uncompressed/AH99/input/'
    QRELS_PATH = '/home/kevinr/IR/'
    DICT_AT_PATH = '/home/kevinr/preference-on-a-budget/pers_script/dict_at/' 
    TREC_EVAL_PATH = 'trec_eval'
    DFIR_PATH = '/home/kevinr/preference-on-a-budget/src/run_as_pickles/overcomplex/AH99.pickle'
    if ARG_preference=='S100':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/S100.csv'
    elif ARG_preference=='ME':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/ME.csv'
    elif ARG_preference=='S2':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/S2.csv'
    elif ARG_preference=='S4':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/S4.csv'
    elif ARG_preference=='moffatratio':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/qrel_pref+rele+ratio-KRNormalized.csv'
    elif ARG_preference=='S100toPref':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/S100toPreference/S100_pairwise_full_scores.csv'
    elif ARG_preference=='MEtoPref':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/S100toPreference/ME_pairwise_full_scores.csv'
    elif ARG_preference=='S2toPref':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/S100toPreference/S2_pairwise_full_scores.csv'
    elif ARG_preference=='S4toPref':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/S100toPreference/S4_pairwise_full_scores.csv'
    elif ARG_preference=='TREC8':
        DFCROWD_PATH = '/home/kevinr/preference-on-a-budget/src/S100.csv'
        
elif ARG_server=='gianluca':
    UNCOMPRESSED_PATH = '/home/kevin/IR/Uncompressed/AH99/input/'
    QRELS_PATH = '/home/kevin/IR/'
    DICT_AT_PATH = '/home/kevin/preference-on-a-budget/pers_script/dict_at/' 
    TREC_EVAL_PATH = 'trec_eval'
    DFIR_PATH = '/home/kevin/preference-on-a-budget/src/run_as_pickles/overcomplex/AH99.pickle'
    if ARG_preference=='S100':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/S100.csv'
    elif ARG_preference=='ME':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/ME.csv'
    elif ARG_preference=='S2':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/S2.csv'
    elif ARG_preference=='S4':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/S4.csv'
    elif ARG_preference=='moffatratio':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/qrel_pref+rele+ratio-KRNormalized.csv'
    elif ARG_preference=='S100toPref':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/S100toPreference/S100_pairwise_full_scores.csv'
    elif ARG_preference=='MEtoPref':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/S100toPreference/ME_pairwise_full_scores.csv'
    elif ARG_preference=='S2toPref':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/S100toPreference/S2_pairwise_full_scores.csv'
    elif ARG_preference=='S4toPref':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/S100toPreference/S4_pairwise_full_scores.csv'
    elif ARG_preference=='TREC8':
        DFCROWD_PATH = '/home/kevin/preference-on-a-budget/src/S100.csv'
    
elif ARG_server=='local':
    UNCOMPRESSED_PATH = '../../../AAA_TREC-DATA/Uncompressed/AH99/input/'
    QRELS_PATH = '../../../AAA_TREC-DATA/TREC-qrels'
    DICT_AT_PATH = './pers_script/dict_at/'
    BASE_PATH_FOLDER = './'
    TREC_EVAL_PATH = 'trec_eval'
    DFIR_PATH = '../../../AAA_TREC-DATA/run_as_pickles/overcomplex/AH99.pickle'

    if ARG_preference=='S100':
        DFCROWD_PATH = '../../../AAA_DATASET/CrowdData/Relevance-Assessment/S100.csv'
    elif ARG_preference=='ME':
        DFCROWD_PATH = '../../../AAA_DATASET/CrowdData/Relevance-Assessment/ME.csv'
    elif ARG_preference=='S2':
        DFCROWD_PATH = '../../../AAA_DATASET/CrowdData/Relevance-Assessment/S2.csv'
    elif ARG_preference=='S4':
        DFCROWD_PATH = '../../../AAA_DATASET/CrowdData/Relevance-Assessment/S4.csv'
    elif ARG_preference=='moffatratio':
        DFCROWD_PATH = '../../../AAA_DATASET/Moffat-and-co/OneDrive_1_25-11-2020/result_files/qrel_pref+rele+ratio.csv'
    elif ARG_preference=='S100toPref':
        DFCROWD_PATH = '../../src/S100toPreference/S100_pairwise_full_scores.csv'
    elif ARG_preference=='S2toPref':
        DFCROWD_PATH = '../../src/S100toPreference/S2_pairwise_full_scores.csv'
    elif ARG_preference=='S4toPref':
        DFCROWD_PATH = '../../src/S100toPreference/S4_pairwise_full_scores.csv'
    elif ARG_preference=='MEtoPref':
        DFCROWD_PATH = '../../src/S100toPreference/ME_pairwise_full_scores.csv'
    elif ARG_preference=='TREC8':
        DFCROWD_PATH = '../../../AAA_DATASET/CrowdData/Relevance-Assessment/S100.csv'
else:
    assert False, 'error on parameter server'

TAG = f'experiment_k-{k}_b-{budget}_which-{ARG_which}_rank-{ARG_rank}_preference-{ARG_preference}_topics-{ARG_topics}_linearize-{ARG_linearize}' 

if ARG_baseline == 1:
    SAVE_PATH = f"./BASELINEexperiments/{TAG}/"
else:
    SAVE_PATH = f"./experiments/{TAG}/"

makedir(f'{SAVE_PATH}/partials/')
makedir(f'{SAVE_PATH}/eval/ours')
makedir(f'{SAVE_PATH}/eval/official')
makedir(f'{SAVE_PATH}/correlations')
makedir(f'{SAVE_PATH}/metrics')
makedir(f'{SAVE_PATH}/qrels')

In [ ]:
dfcrowd = pd.read_csv(f'{DFCROWD_PATH}')
if  ARG_preference=='S100':
    dfcrowd = dfcrowd[['doc_id','S100_worker_id','S100_rel']]
    dfcrowd.columns = ['doc_id','worker_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='ME':
    dfcrowd = dfcrowd[['doc_id','ME_worker_id','ME_rel']]
    dfcrowd.columns = ['doc_id','worker_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='S2':
    dfcrowd = dfcrowd[['doc_id','S2_worker_id','S2_rel']]
    dfcrowd.columns = ['doc_id','worker_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='S4':
    dfcrowd = dfcrowd[['doc_id','S4_worker_id','S4_rel']]
    dfcrowd.columns = ['doc_id','worker_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='moffatratio':
    dfcrowd = dfcrowd[['topic','doc','ratio']]
    dfcrowd.columns = ['topic','doc_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='S100toPref':
    dfcrowd = dfcrowd[['topic','document','score']]
    dfcrowd.columns = ['topic','doc_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='MEtoPref':
    dfcrowd = dfcrowd[['topic','document','score']]
    dfcrowd.columns = ['topic','doc_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='S2toPref':
    dfcrowd = dfcrowd[['topic','document','score']]
    dfcrowd.columns = ['topic','doc_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='S4toPref':
    dfcrowd = dfcrowd[['topic','document','score']]
    dfcrowd.columns = ['topic','doc_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
if  ARG_preference=='TREC8':
    dfcrowd = dfcrowd[['topic','S100_worker_id','rel_TREC']]
    dfcrowd.columns = ['topic','doc_id','rel']
    dfcrowd.set_index('doc_id', inplace=True)
    dfcrowd.fillna(0, inplace=True)

In [ ]:
if ARG_topics=='all':
    topics_to_consider = topics_to_consider
if ARG_topics=='subsetmoffat':
    topics_to_consider = topics_to_consider_moffat[:]
        
topics = topics_to_consider

In [ ]:
dfirs = pd.read_pickle(f'{DICT_AT_PATH}/AH99_dict_at_{k}.pickle')
dfir = pd.read_pickle(f'{DFIR_PATH}')#.reset_index()
dfir = dfir.loc[dfir.index.get_level_values('topic').isin(topics_to_consider)]

In [ ]:
"""--- select which couples to judge --- """

def which_all(i,topic):
    global dfirs
    sub_dfirs = dfirs[topic]
    docs = list(sub_dfirs.keys())
    docs = list(itertools.combinations(docs,2))
    
    save_pickle(docs,f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')
    return   

def which_adjacent(i,topic):
    order_couples = load_pickle(f'./pers_script/adj_couples/CouplesToJudgeADJ_{k}_{topic}.pkl')  
    save_pickle(order_couples,f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')
    return   

def which_alleverysystem(i,topic):
    order_couples = load_pickle(f'./pers_script/all_couples_every_system/AllCouplesEverySystem_{k}_{topic}.pkl')  
    save_pickle(order_couples,f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')
    return   


#['all', 'adjacent', 'alleverysystem']
print(f'\t computing which couples to judge {ARG_which}')
pool = mp.Pool(min(mp.cpu_count(),8))
args = [(i,topic) for i,topic in enumerate(topics)]
if ARG_which == 'all':
    results = pool.starmap(which_all, args)
elif ARG_which == 'adjacent':
    results = pool.starmap(which_adjacent, args)
elif ARG_which == 'alleverysystem':
    results = pool.starmap(which_alleverysystem, args)
else:
    assert False, 'error on arg document score'
pool.close()    
print(f'\t done computing which couples to judge {ARG_which}')

In [ ]:
###### """--- rank all couples and cut to budget --- """

def rank_avgrank(i,topic):
    global budget, k, dfirs
    # load couples
    pool_couples = load_pickle(f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')
    # compute score for each document
    sub_dfirs = dfirs[topic]
    docs = list(sub_dfirs.keys())
    doc_score = {}
    for d in docs:
        ar_d = np.average(sub_dfirs[d])
        p_d = (k-ar_d)/k
        doc_score[d] = p_d
    # assign score to each couple
    couples_score = {}
    for d1, d2 in pool_couples:
        score_d1 = doc_score.get(d1,-1)
        score_d2 = doc_score.get(d2,-1)
        score = 1-np.abs(score_d1-score_d2)
        couples_score[(d1,d2)] = score
    # sort
    couples_score = [k for k,v in sorted(couples_score.items(), key=itemgetter(1))]
    # cut at budget 
    couples_score = couples_score[:budget]
    save_pickle(couples_score,f'./{SAVE_PATH}/partials/Rank_{k}_{topic}.pkl')
    return

def rank_borda(i,topic):
    global budget, k, dfirs
    # load couples
    pool_couples = load_pickle(f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')
    # compute score for each document
    sub_dfirs = dfirs[topic]
    docs = list(sub_dfirs.keys())
    doc_score = {}
    for d in docs:
        ar_d = np.average(sub_dfirs[d])
        p_d = (k-ar_d)/k
        doc_score[d] = p_d
    # assign score to each couple
    couples_score = {}
    for d1, d2 in pool_couples:
        score_d1 = doc_score.get(d1,-1)
        score_d2 = doc_score.get(d2,-1)
        score = 1-np.abs(score_d1-score_d2)
        couples_score[(d1,d2)] = score
    # sort
    couples_score = [k for k,v in reversed(sorted(couples_score.items(), key=itemgetter(1)))]
    # cut at budget 
    couples_score = couples_score[:budget]
    save_pickle(couples_score,f'./{SAVE_PATH}/partials/Rank_{k}_{topic}.pkl')
    return
    
def rank_none(i,topic):
    # load couples
    pool_couples = load_pickle(f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')    
    save_pickle(pool_couples,f'./{SAVE_PATH}/partials/Rank_{k}_{topic}.pkl')
    return

def rank_avgrankreversed(i,topic):
    global budget, k, dfirs
    # load couples
    pool_couples = load_pickle(f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')
    # compute score for each document
    sub_dfirs = dfirs[topic]
    docs = list(sub_dfirs.keys())
    doc_score = {}
    for d in docs:
        ar_d = np.average(sub_dfirs[d])
        p_d = (k-ar_d)/k
        doc_score[d] = p_d
    # assign score to each couple
    couples_score = {}
    for d1, d2 in pool_couples:
        score_d1 = doc_score.get(d1,-1)
        score_d2 = doc_score.get(d2,-1)
        score = 1-np.abs(score_d1-score_d2)
        couples_score[(d1,d2)] = score
    # sort
    couples_score = [k for k,v in reversed(sorted(couples_score.items(), key=itemgetter(1)))]
    # cut at budget 
    couples_score = couples_score[:budget]
    save_pickle(couples_score,f'./{SAVE_PATH}/partials/Rank_{k}_{topic}.pkl')
    return
  
    
def rank_rff(i,topic):
    global budget, k, dfirs
    # load couples
    pool_couples = load_pickle(f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')
    # compute score for each document
    sub_dfirs = dfirs[topic]
    docs = list(sub_dfirs.keys())
    doc_score = {}
    for d in docs:
        p_d = np.sum([1/(0+x) for x in sub_dfirs[d]])
        doc_score[d] = p_d
    # assign score to each couple
    couples_score = {}
    for d1, d2 in pool_couples:
        score_d1 = doc_score.get(d1,-1)
        score_d2 = doc_score.get(d2,-1)
        score = 1-np.abs(score_d1-score_d2)
        couples_score[(d1,d2)] = score
    # sort
    couples_score = [k for k,v in sorted(couples_score.items(), key=itemgetter(1))]
    # cut at budget 
    couples_score = couples_score[:budget]
    save_pickle(couples_score,f'./{SAVE_PATH}/partials/Rank_{k}_{topic}.pkl')
    return

def rank_combMNZ(i,topic):
    global budget, k, dfirs,dfirsscores
    # load couples
    pool_couples = load_pickle(f'{SAVE_PATH}/partials/Which_{k}_{topic}.pkl')
    # compute score for each document
    sub_dfirs = dfirs[topic]
    sub_dfirsscores = dfirsscores[topic]
    
    docs = list(sub_dfirs.keys())
    doc_score = {}
    for d in docs:
        num_pos = len(set(sub_dfirs[d]))
        p_d = np.sum(sub_dfirsscores[d])
        p_d = p_d * num_pos
        doc_score[d] = p_d
    # assign score to each couple
    couples_score = {}
    for d1, d2 in pool_couples:
        score_d1 = doc_score.get(d1,-1)
        score_d2 = doc_score.get(d2,-1)
        score = 1-np.abs(score_d1-score_d2)
        couples_score[(d1,d2)] = score
    # sort
    couples_score = [k for k,v in sorted(couples_score.items(), key=itemgetter(1))]
    # cut at budget 
    couples_score = couples_score[:budget]
    save_pickle(couples_score,f'./{SAVE_PATH}/partials/Rank_{k}_{topic}.pkl')
    return
    
#['none', 'avgrank', 'borda', 'rff' ,'avgrankreversed']
if ARG_rank == 'combMNZ':
    dfirsscores = pd.read_pickle(f'{DICT_AT_PATH}/AH99_dict_at_{k}_rel.pickle')

print(f'\t computing rank {ARG_rank}')
pool = mp.Pool(min(mp.cpu_count(),8))
args = [(i,topic) for i,topic in enumerate(topics)]
if ARG_rank == 'none':
    results = pool.starmap(rank_none, args)
elif ARG_rank == 'avgrank':
    results = pool.starmap(rank_avgrank, args)
elif ARG_rank == 'borda':
    results = pool.starmap(rank_borda, args)
elif ARG_rank == 'rrf':
    results = pool.starmap(rank_rff, args)
elif ARG_rank == 'combMNZ':
    results = pool.starmap(rank_combMNZ, args)
elif ARG_rank == 'avgrankreversed':
    results = pool.starmap(rank_avgrankreversed, args)
else:
    assert False, 'error on arg rank couple'
pool.close()    
print(f'\t done computing rank {ARG_rank}')

In [ ]:
"""--- compute preferences -- take average judgment by crowd for all couples to judge --- """

if ARG_preference in ['S100', 'ME', 'S2', 'S4', 'moffatratio','TREC8']:
    subdfcrowd = dfcrowd.reset_index()
    subdfcrowd = subdfcrowd.groupby(['doc_id']).agg('mean').reset_index()
    subdfcrowd = subdfcrowd[['doc_id','rel']]
    docs = subdfcrowd['doc_id'].values
    rels = subdfcrowd['rel'].values
    subdfcrowd = {docs[x]:rels[x] for x in range(len(docs))}
elif ARG_preference in ['S100toPref','MEtoPref','S2toPref','S4toPref']:
    subdfcrowd = dfcrowd.copy()
    subdfcrowd['topic'] = subdfcrowd['topic'].astype(str)
    subdfcrowd.set_index('topic', inplace=True)
else:
    assert False, "Error on preference_crowd"
    
def preference_crowd(i,topic):
    global  k
    sub_couples = load_pickle(f'./{SAVE_PATH}/partials/Rank_{k}_{topic}.pkl')
    
    #sanitize for document rename
    sub_couples = [ (d1.split("_")[0],d2.split("_")[0]) if ("_" in d1) else (d1,d2) for d1,d2 in sub_couples]
    
    #rank by average value
    crowd_couples = [(d1, d2, subdfcrowd.get(d1,-1), subdfcrowd.get(d2,-1)) for d1,d2 in sub_couples]
    
    # compute preference
    crowd_couples = [(d1,d2,d1) if v1>=v2 else (d1,d2,d2) for d1,d2,v1,v2 in crowd_couples]
    
    save_pickle(crowd_couples,f'./{SAVE_PATH}/partials/Preference_{k}_{topic}.pkl')
    return 

def preference_crowdpref(i,topic):
    global  k,subdfcrowd
    sub_couples = load_pickle(f'./{SAVE_PATH}/partials/Rank_{k}_{topic}.pkl')
    
    #sanitize for document rename
    sub_couples = [ (d1.split("_")[0],d2.split("_")[0]) if ("_" in d1) else (d1,d2) for d1,d2 in sub_couples]
        
    crowd_couples = subdfcrowd.loc[str(topic)]
    crowd_couples = crowd_couples[['first', 'second', 'winner']]
    crowd_couples = [tuple(x) for x in crowd_couples.to_numpy()]
    
    #crowd_couples = [(x,y,z) for x,y,z in crowd_couples if ((x,y) in sub_couples) or ((y,x) in sub_couples)]   
    
    save_pickle(crowd_couples,f'./{SAVE_PATH}/partials/Preference_{k}_{topic}.pkl')
    return 


print(f'\t computing preference {ARG_preference}')
pool = mp.Pool(min(mp.cpu_count(),8))
args = [(i,topic) for i,topic in enumerate(topics)]

if ARG_preference in ['S100', 'ME', 'S2', 'S4', 'moffatratio','S100toPref','MEtoPref','S2toPref','S4toPref','TREC8']:
    results = pool.starmap(preference_crowd, args)
elif ARG_preference in ['nonefornow']:
    results = pool.starmap(preference_crowdpref, args)
else:
    assert False, "Error on preference_crowd"

pool.close()    
print(f'\t done computing preference {ARG_preference}')

In [ ]:
"""--- linearize preferences  --- """
from mytoposort import comp_winlose,sort_by_wins
from linearize import linearize

def linearize_sortbywin(i,topic):
    global k
    preferences =  load_pickle(f'./{SAVE_PATH}/partials/Preference_{k}_{topic}.pkl')
    partial_order = comp_winlose(preferences)
    partial_order = sort_by_wins(partial_order)
    scores = [i for i in range(len(partial_order))][::-1]
    res = {}
    res['scores'] = scores
    res['partial_order'] = partial_order
    save_pickle(res,f'./{SAVE_PATH}/partials/Linearize_{k}_{topic}.pkl')
    return 

def linearize_BTL(i,topic):
    global k
    preferences =  load_pickle(f'./{SAVE_PATH}/partials/Preference_{k}_{topic}.pkl')
    #print(preferences)
    assert len(preferences)>0, f"error on len for {topic}"
    partial_order,scores = linearize(preferences)
    scores = [i for i in range(len(partial_order))][::-1]
    res = {}
    res['scores'] = scores
    res['partial_order'] = partial_order
    save_pickle(res,f'./{SAVE_PATH}/partials/Linearize_{k}_{topic}.pkl')
    return 

def linearize_BTLScores(i,topic):
    global k
    preferences =  load_pickle(f'./{SAVE_PATH}/partials/Preference_{k}_{topic}.pkl')
    assert len(preferences)>0, "error on len"
    partial_order,scores = linearize(preferences)
    res = {}
    res['scores'] = scores
    res['partial_order'] = partial_order
    save_pickle(res,f'./{SAVE_PATH}/partials/Linearize_{k}_{topic}.pkl')
    return 

# linearize_BTL(0,442)
# assert False

print(f'\t computing linearize {ARG_linearize}')
pool = mp.Pool(min(mp.cpu_count(),8))
args = [(i,topic) for i,topic in enumerate(topics)]

if ARG_linearize == 'sortbywin':
    results = pool.starmap(linearize_sortbywin, args)
elif ARG_linearize == 'btl':
    results = pool.starmap(linearize_BTL, args)
elif ARG_linearize == 'btlscores':
    results = pool.starmap(linearize_BTLScores, args)    
else:
    assert False, 'error on arg rank couple'
pool.close()    
print(f'\t done computing linearize {ARG_linearize}')

In [ ]:
"""---compute qrels and eval  --- """

def linearizenotthrow(lista_ties, lista_off, scores):
    def move_to_top(mylist, targetvalue):
        res = mylist.copy()
        res.insert(0, res.pop(res.index(targetvalue)))
        return res
    lista_off = list(lista_off)
    lista_lin = []
    lista_values = []
    if len(lista_ties)==1:
        lista_ties = [lista_ties]
    for i,x in enumerate(lista_ties):
        if len(x)==1:
            lista_lin.append(x[0])
            lista_values.append(scores[i])
        else:
            for elem in x:
                 if elem in lista_off:
                    x = move_to_top(x,elem)

            for elem in x:
                if elem in lista_off:
                    lista_values = [x+1 for x in lista_values]
                    lista_lin.append(elem)
                    lista_values.append(scores[i]+1)
                else:
                    lista_lin.append(elem)
                    lista_values.append(scores[i])
    lista_values = [x+1 for x in lista_values]

    return lista_lin, lista_values

def compute_qrel_and_eval(z,run):
    global dfir, k, topics
    
    df_qrel = []
    for topic in topics:
        #print(run, topic)
        run_docs = dfir.loc[run].loc[topic]['list']['doc'].values[:k]
        pos = load_pickle(f'./{SAVE_PATH}/partials/Linearize_{k}_{topic}.pkl')
        scores = pos['scores']
        po = pos['partial_order']
                
        linear_list, values_list = linearizenotthrow(lista_ties=po, lista_off=run_docs, scores=scores)
        
        for i in range(len((linear_list))):
            df_qrel.append( (topic,0,linear_list[i],values_list[i])  )
        
    with open(f'./{SAVE_PATH}/qrels/qrels_{k}_{run}.txt', 'w') as fp:
        fp.write(f'\n'.join('{} {} {} {}'.format(x[0],x[1],x[2],x[3]) for x in df_qrel))
    
    os.system(f"{TREC_EVAL_PATH} -q -m ndcg -M {k} {SAVE_PATH}/qrels/qrels_{k}_{run}.txt  {UNCOMPRESSED_PATH}/input.{run} > ./{SAVE_PATH}/eval/ours/{run}.txt ")
    time.sleep(0.1)
    os.system(f"{TREC_EVAL_PATH} -q -m ndcg -M {k} {QRELS_PATH}/qrels.AH99.txt {UNCOMPRESSED_PATH}/input.{run} > ./{SAVE_PATH}/eval/official/{run}.txt ")
    time.sleep(0.1)
    return 
import time
runs = np.unique(dfir.index.get_level_values('run'))

print('\t computing qrels and evaluating')
pool = mp.Pool(min(mp.cpu_count(),8))
#pool = mp.Pool(1)

args = [(i,run)for i,run in enumerate(runs)]
results = pool.starmap(compute_qrel_and_eval, args)
pool.close()    
#results = [item for sublist in results for item in sublist]
print('\t done computing qrels and evaluating')

In [ ]:
PATH = f"{SAVE_PATH}/qrels/"
files = [f for f in listdir(PATH) if (isfile(join(PATH, f))) & (f != ".DS_Store")&("qrels_" in f)&("orcl99man" in f)]

def read_specific(DATASET,budget):
    dfev = pd.DataFrame(columns=['k','b','which','rank','preference','topics','linearize','kind','topic', 'system', 'metric','score'])
    PATH = f'./server_experiments/{DATASET}/'
    files = [f for f in listdir(PATH) if (isfile(join(PATH, f))) & (f != ".DS_Store")&("correlation" not in f)&(f"{budget}" in f)]

    for f in files:#, desc='files', leave=False):
        k = f.split("_")[2].split("-")[1]
        b = f.split("_")[3].split("-")[1]
        which = f.split("_")[4].split("-")[1]
        rank = f.split("_")[5].split("-")[1]
        preference = f.split("_")[6].split("-")[1]
        topics = f.split("_")[7].split("-")[1]
        linearize = f.split("_")[8].split("-")[1].split(".")[0]

        if isfile(f'./{PATH}/{f}'):
            cors = pd.read_csv(f'{PATH}/{f}')
            cors['k'] = k
            cors['b'] = b
            cors['which'] = which
            cors['rank'] = rank
            cors['preference'] = preference
            cors['topics'] = topics
            cors['linearize'] = linearize
            cors = cors[dfev.columns]
            
            dfev = pd.concat([dfev,cors])
            
    dfev['k'] = dfev['k'].astype(int)
    dfev['b'] = dfev['b'].astype(int)
    dfev['score'] = dfev['score'].astype(float)
    dfev['label'] = dfev['which'] + "-" + dfev['rank'] + "-" + dfev['linearize']
    
    return dfev

cors = read_specific('TREC8',budget)
cors = cors[(cors['linearize']==ARG_linearize)
           &(cors['which']==ARG_which)
           &(cors['rank']==ARG_rank)]
cors = cors.groupby(['kind','metric','system']).agg('mean').reset_index()


for f in tqdm(files, desc='files', leave=False):
    sysname = f.replace('qrels_10_','').replace('.txt','')
    
    subcors_trec =  cors[(cors['kind']=='TREC8')&(cors['system']==sysname)]['score'].values[0]
    subcors_ours =  cors[(cors['kind']=='ours')&(cors['system']==sysname)]['score'].values[0]
    

    our_qrels = pd.read_csv(f"{SAVE_PATH}/qrels/{f}", sep=' ', header=None)
    our_qrels.columns = ['topic','zero','doc','score_our']
    our_qrels = our_qrels[['topic','doc','score_our']]

    off_qrels = pd.read_csv(f"{QRELS_PATH}/qrels.AH99.txt", sep=' ', header=None)
    off_qrels.columns = ['topic','zero','doc','score_off']
    off_qrels = off_qrels[['topic','doc','score_off']]

    merged = our_qrels.merge(off_qrels, how='inner', on=['topic','doc'])

    # display(our_qrels.head())
    # display(off_qrels.head())
    #display(merged.head())

    import seaborn as sns 
    import pylab as plt
    from matplotlib.backends.backend_pdf import PdfPages
    pd.set_option('display.max_columns', 500)
    %matplotlib inline

    sns.set_style('whitegrid')
    plt.rcParams['figure.figsize']=(7,3)

    import matplotlib
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42

    #merged = merged.head(10)
    

    pages = PdfPages(f"../../plots/qrels/{sysname}.pdf")
    
    fig, axs = plt.subplots(ncols=2, nrows=1)
    
    sns.boxplot(x="score_off", y="score_our", data=merged,
                     boxprops=dict(facecolor='#FFFFFF', alpha=1, zorder = 0), ax=axs[0])

    sns.violinplot(x="score_off", y="score_our", data=merged,
                     inner='quartile', ax=axs[0])
    
    
    x = cors[cors['kind']=="TREC8"]['score'].values
    y = cors[cors['kind']=="ours"]['score'].values
    
    sns.regplot(x,y, fit_reg=True, ax=axs[1])

    
    sns.regplot([subcors_trec],[subcors_ours], fit_reg=False, ax=axs[1], color='red', marker='s')
    
    axs[1].set(xlim=(-0.05, 0.3),ylim=(-0., 0.25))
    #plt.yticks([0.1, 0.2])
    
    axs[0].set(xlabel=f"official",ylabel=f"pairwise")
    pages.savefig(bbox_inches='tight');pages.close()
    plt.close()
    assert False

In [ ]:
"""--- evaluate  --- """

df_ev = pd.DataFrame(columns=['kind','topic', 'system', 'metric','score'])

def parses(path, kind):
    df_ev = pd.DataFrame(columns=['kind','topic', 'system', 'metric','score'])
    RUN_PATH = path
    files = [f for f in listdir(RUN_PATH) if (isfile(join(RUN_PATH, f))) & (f != ".DS_Store")]
    for i, f in enumerate(files):
        #print(f)
        sys = f.replace('summary.{}.input.'.format(kind), '').replace('.txt', '')
        df =  pd.read_csv(join(RUN_PATH, f), sep='\t', header=None)
        df.columns = ['metric', 'topic', 'score']
        df['metric'] = [x.strip() for x in df['metric']]
        df = df[df['metric']=='ndcg']
        df = df[df['topic']!='all']
        df['kind'] = kind
        df['metric'] = 'ndcg'
        df['system'] = sys
               
        df_ev = pd.concat([df_ev, df])
        
    return df_ev

df_eval = pd.concat([
parses(f'./{SAVE_PATH}/eval/official/', 'TREC8'),
parses(f'./{SAVE_PATH}/eval/ours/', f'ours'),
])

df_eval = df_eval[df_eval['topic'].isin(topics)]
df_eval['score'] = df_eval['score'].astype(float)
df_eval.to_csv(f'{SAVE_PATH}/metrics/metric_{TAG}.csv', index=False)

In [ ]:
'''save correlation'''

#from rbo import rbo
from tauAP import tauAP, rank
import scipy.stats


ddres = pd.DataFrame(columns=['kind','x','y','rho','tau', 'tauAP','tau10'])

for kind in ['topics-all','topics-moffat']:
    if kind == 'topics-all':
        df_plot = df_eval.groupby(['kind', 'system','metric']).agg('mean').reset_index()
    else:
        df_plot = df_eval[df_eval['topic'].isin(topics_to_consider_moffat)].groupby(['kind', 'system','metric']).agg('mean').reset_index()

    ords = [('TREC8',f'ours')]
    for i, (x_val, y_val) in enumerate(ords):
        x = df_plot[df_plot['kind']==x_val]['score'].values 
        y = df_plot[df_plot['kind']==y_val]['score'].values

        try:
            rho, rhoval = sp.stats.pearsonr(x,y)
            tau, tauval = sp.stats.kendalltau(x,y)

            tauap = tauAP(x,y, top_heavy=True)

            xten = x
            yten = y
            xten, yten = zip(*sorted(zip(xten, yten)))
            xten = list(xten[::-1])
            yten = list(yten[::-1])
            xten = xten[:10]
            yten = yten[:10]
            tauten, tauvalten = sp.stats.kendalltau(xten,yten)      

            ddres.loc[len(ddres)] = [kind,x_val, y_val, rho, tau, tauap, tauten]
        except:
            ddres.loc[len(ddres)] = [kind,x_val, y_val, np.nan, np.nan, np.nan, np.nan]
    #ddres
    ddres.to_csv(f'./{SAVE_PATH}/correlations/correlation_{TAG}.csv', index=False)
    print(ddres)

In [ ]:
'''delete folders'''
import shutil
rmdir(f"{SAVE_PATH}/eval")
rmdir(f"{SAVE_PATH}/partials")
rmdir(f"{SAVE_PATH}/qrels")


In [ ]:
# " ".join([str(x) for x in range(10, 101, 10)]) +" "+ \
# " ".join([str(x) for x in range(100, 5001, 100)]) +" "+ \
# " ".join([str(x) for x in range(10000, 100001, 10000)])